In [1]:
import numpy as np
import pandas as pd
import cv2
from datetime import datetime

In [2]:
moves_df = pd.read_csv("../data/overhang/bae8f52c-407e-5f89-a8e3-61fcca51ee0a_moves.csv")

In [3]:
moves_df.head()

,hand,climb_index,move_index,start_time,end_time,climb,on_hold,duration
0,left,0,0,2021-02-25 13:45:57.460000038+00:00,2021-02-25 13:45:58.559999943+00:00,0,0.44,1.1
1,right,0,0,2021-02-25 13:46:01.960000038+00:00,2021-02-25 13:46:02.960000038+00:00,0,4.94,1.0
2,left,0,1,2021-02-25 13:46:04.559999943+00:00,2021-02-25 13:46:05.859999895+00:00,0,6.00,1.3
3,right,0,1,2021-02-25 13:46:06.559999943+00:00,2021-02-25 13:46:07.559999943+00:00,0,3.60,1.0
4,left,0,2,2021-02-25 13:46:08.660000086+00:00,2021-02-25 13:46:09.559999943+00:00,0,2.80,0.9


In [4]:
moves_df[['start_time', 'end_time']] = moves_df[['start_time', 'end_time']].apply(pd.to_datetime)

In [5]:
cap = cv2.VideoCapture("../data/overhang/Overhang (vertical + horizontal).mp4")

ret, frame1 = cap.read()
ret, frame2 = cap.read()

fps = cap.get(cv2.cv2.CAP_PROP_FPS)
print(fps)

29.99390600005078


In [6]:
def time_to_frame(end_time):
    t_step = end_time.replace(tzinfo=None)  - moves_df.start_time[0].replace(tzinfo=None) 
    frame_number = t_step.total_seconds() * fps
    return frame_number

In [7]:
moves_df['frame_number'] = moves_df['end_time'].apply(time_to_frame)

In [8]:
moves_df.head(10)

,hand,climb_index,move_index,start_time,end_time,climb,on_hold,duration,frame_number
0,left,0,0,2021-02-25 13:45:57.460000038+00:00,2021-02-25 13:45:58.559999943+00:00,0,0.44,1.1,32.993267
1,right,0,0,2021-02-25 13:46:01.960000038+00:00,2021-02-25 13:46:02.960000038+00:00,0,4.94,1.0,164.966483
2,left,0,1,2021-02-25 13:46:04.559999943+00:00,2021-02-25 13:46:05.859999895+00:00,0,6.00,1.3,251.948780
3,right,0,1,2021-02-25 13:46:06.559999943+00:00,2021-02-25 13:46:07.559999943+00:00,0,3.60,1.0,302.938421
4,left,0,2,2021-02-25 13:46:08.660000086+00:00,2021-02-25 13:46:09.559999943+00:00,0,2.80,0.9,362.926233
5,right,0,2,2021-02-25 13:46:10.059999943+00:00,2021-02-25 13:46:11.059999943+00:00,0,2.50,1.0,407.917092
6,left,0,3,2021-02-25 13:46:13.759999990+00:00,2021-02-25 13:46:14.960000038+00:00,0,4.20,1.2,524.893355
7,right,0,3,2021-02-25 13:46:14.160000086+00:00,2021-02-25 13:46:15.859999895+00:00,0,3.10,1.7,551.887840
8,right,0,4,2021-02-25 13:46:16.259999990+00:00,2021-02-25 13:46:17.460000038+00:00,0,0.40,1.2,599.878120
9,left,0,4,2021-02-25 13:46:19.759999990+00:00,2021-02-25 13:46:21.059999943+00:00,0,4.80,1.3,707.856152


In [12]:
moves_df.start_time[0].replace(tzinfo=None)

Timestamp('2021-02-25 13:45:57.460000038')

In [13]:
moves_df.end_time.values[-1]

numpy.datetime64('2021-02-25T13:49:59.259999990')

In [15]:
t = moves_df.end_time.values[0] - moves_df.start_time[0].replace(tzinfo=None)
t.total_seconds()

1.099999

In [26]:
frame_counter = 100
moves_df = [moves_df['frame_number'] <= frame_counter]

In [ ]:
cap = cv2.VideoCapture("../data/overhang/Overhang (vertical + horizontal).mp4")

ret, frame1 = cap.read()
ret, frame2 = cap.read()

fps = cap.get(cv2.cv2.CAP_PROP_FPS)
print(fps)

# We need to set resolutions.
# so, convert them from float to integer.
width= int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height= int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))


# Below VideoWriter object will create 
# a frame of above defined The output  
# is stored in 'filename.avi' file. 
result= cv2.VideoWriter('moves_tracker.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 30, (width,height))

frame_counter = 0
# Read the video
while(cap.isOpened()):
    # Subtract current frame from the previous 
    diff = cv2.absdiff(frame1, frame2)
    
    # Convert to grayscale
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    canny_high = cv2.Canny(gray, 250, 500)
    _, thresh = cv2.threshold(canny_high, 20, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    dilated = cv2.dilate(thresh, None, iterations=10)
    contours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours:
        (x, y, w, h) = cv2.boundingRect(contour)
        
        if cv2.contourArea(contour) < 700:
            continue
            
        cv2.rectangle(frame1, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame1, "Status: {}".format('Movement'), (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
    
    if frame_counter
    
    
    # Stacking the images to print them together
    # For comparison
    #images = np.hstack((gray, dilated1, dilated2))
    result.write(frame1)
    cv2.imshow('frame',frame1)
    frame1 = frame2
    ret, frame2 = cap.read()
    frame_counter += 1
    
    # Press Q on keyboard to  exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
result.release()
cv2.destroyAllWindows()

29.99390600005078


In [13]:
cap = cv2.VideoCapture("test.mp4")

ret, frame1 = cap.read()
ret, frame2 = cap.read()

# We need to set resolutions. 
# so, convert them from float to integer. 
frame_width = int(cap.get(3)) 
frame_height = int(cap.get(4)) 
   
size = (frame_width, frame_height) 
   
# Below VideoWriter object will create 
# a frame of above defined The output  
# is stored in 'filename.avi' file. 
result = cv2.VideoWriter('filename.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 20.0, (640,480))

# Read the video
while(cap.isOpened()):
    # Subtract current frame from the previous 
    diff = cv2.absdiff(frame1, frame2)
    
    # Convert to grayscale
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    #blur = cv2.GaussianBlur(gray, (7,7), 0)
    canny = cv2.Canny(gray, 20, 30)
    canny_high = cv2.Canny(gray, 250, 500)
    
    _, thresh1 = cv2.threshold(canny, 20, 255, cv2.THRESH_BINARY)
    _, thresh2 = cv2.threshold(canny_high, 20, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    
    dilated1 = cv2.dilate(thresh1, None, iterations=3)
    dilated2 = cv2.dilate(thresh2, None, iterations=10)
    
    contours1, _ = cv2.findContours(dilated1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours2, _ = cv2.findContours(dilated2, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours2:
        (x, y, w, h) = cv2.boundingRect(contour)
        
        if cv2.contourArea(contour) < 700:
            continue
            
        cv2.rectangle(frame1, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame1, "Status: {}".format('Movement'), (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
    
    
    # Stacking the images to print them together
    # For comparison
    #images = np.hstack((gray, dilated1, dilated2))
    result.write(frame1)
    cv2.imshow('frame',frame1)
    frame1 = frame2
    ret, frame2 = cap.read()
    
    # Press Q on keyboard to  exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
result.release()
cv2.destroyAllWindows()

error: OpenCV(4.5.1) /tmp/pip-req-build-ms668fyv/opencv/modules/core/src/arithm.cpp:666: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and the same number of channels), nor 'array op scalar', nor 'scalar op array' in function 'arithm_op'


In [ ]:
cap = cv2.VideoCapture("test.mp4")

ret, frame1 = cap.read()
ret, frame2 = cap.read()

while(cap.isOpened()):
    diff = cv2.absdiff(frame1, frame2)
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    _, thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
    dilated = cv2.dilate(thresh, None, iterations=3)
    contours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
    for contour in contours:
        (x, y, w, h) = cv2.boundingRect(contour)
        
        if cv2.contourArea(contour) < 700:
            continue
            
        cv2.rectangle(frame1, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame1, "Status: {}".format('Movement'), (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
        
    
    #cv2.drawContours(frame1, contours, -1, (0, 255, 0), 2)
    
    cv2.imshow('feed',frame1)
    frame1 = frame2
    ret, frame2 = cap.read()
    
    if cv2.waitKey(40) == 27:
        break
        
    if cv2.getWindowProperty('frame', cv2.WND_PROP_VISIBLE) < 1:
        break

cap.release()
cv2.destroyAllWindows()